# Flight Delays: Web App

Here we will prepare data for vizualizations in the web app. Because we have over 14,000 records it will be useful to prep and save some of the data into a smaller size by stripping it down to the bare essentials we're trying to vizualize.

In [17]:
import pandas as pd
import glob
import os
import requests
import json
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import sqlite3 as db
import datetime
from pytz import timezone
import pytz

from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

import pandas as pd
import glob
import os
import requests
import json
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,\
precision_recall_fscore_support, f1_score, plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_validate, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import random
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import xgboost as xgb

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay

from keras import regularizers
from keras import models
from keras import layers
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight

import keras
from keras.models import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
from scipy import stats
import pickle

### Business Goals

Here we want to achieve the following:

1. Prepare data for vizualizations by date
2. Prepare data for vizualizations by airline

## Importing data

In [40]:
carrier_data = pd.read_csv('data/prepared/data_for_graphing.csv')

In [41]:
relevant_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'CLT', 'LAS', 'PHX', 
                     'MCO', 'SEA', 'MIA', 'IAH', 'JFK', 'FLL', 'EWR', 'SFO', 'MSP', 'DTW',
                     'BOS', 'SLC', 'PHL', 'BWI', 'TPA', 'SAN', 'MDW', 'LGA', 'BNA', 'IAD',
                     'DAL', 'DCA', 'PDX', 'AUS', 'HOU', 'HNL', 'STL', 'RSW', 'SMF', 'MSY',
                     'SJU', 'RDU', 'OAK', 'MCI', 'CLE', 'IND', 'SAT', 'SNA', 'PIT', 'CVG',
                     'CMH', 'PBI', 'JAX', 'MKE', 'ONT', 'ANC', 'BDL', 'OGG', 'OMA', 'MEM',
                     'BOI', 'RNO', 'CHS', 'OKC']

airport_filter = '|'.join(relevant_airports)

carrier_data = carrier_data[carrier_data['ORIGIN'].str.contains(airport_filter)]

In [11]:
carrier_data['airport-lookup-key'] = carrier_data['ORIGIN'] + '-' + carrier_data['DEST']
airport_lookup = carrier_data.drop_duplicates(subset=['airport-lookup-key'])

In [14]:
airport_lookup = airport_lookup[['ORIGIN', 'DEST','airport-lookup-key', 'origin-elevation','dest-elevation', 'DISTANCE','dest-lat-long','origin-lat-long','origin-tz','dest-tz']]

In [18]:
airport_lookup.to_csv('data/prepared/airport_lookup.csv', index=False)

## Grouping Data by Day

To group by day, we'll need to create a datetime field to group by first.

In [48]:
carrier_data['severe_delay'] = carrier_data['ARR_DELAY_NEW'] > 60
carrier_data['severe_delay'] = carrier_data['severe_delay'].map({True: 'Severe Delays', False: 'No Severe Delays'})

Next, we want to rename those airlines to something more user-friendly.

In [5]:
df['FL_DATE_LOCAL_ROUNDED'] = pd.to_datetime(df['FL_DATE_LOCAL_ROUNDED'])
df['rounded-hour'] = df['FL_DATE_LOCAL_ROUNDED'].dt.hour.astype(int)

In [6]:
df['FL_DATE_LOCAL'] = pd.to_datetime(df['FL_DATE_LOCAL'])
df['FL_DATE_LOCAL'] = df['FL_DATE_LOCAL'].dt.date

In [7]:
conditions = [
    (df['rounded-hour'] >= 5) & (df['rounded-hour'] <= 8),
    (df['rounded-hour'] >= 8) & (df['rounded-hour'] <= 12),
    (df['rounded-hour'] > 12) & (df['rounded-hour'] <= 15),
    (df['rounded-hour'] > 15) & (df['rounded-hour'] <= 17),
    (df['rounded-hour'] > 17) & (df['rounded-hour'] <= 19),
    (df['rounded-hour'] > 19) & (df['rounded-hour'] <= 21),
    (df['rounded-hour'] >= 0) & (df['rounded-hour'] < 5),
    (df['rounded-hour'] > 21)
]
values = ['Early Morning', 'Late Morning', 'Early Afternoon', 'Late Afternoon', 'Early Evening',  'Late Evening', 'Night', 'Night']
df['Time of Day'] = np.select(conditions, values)

In [46]:
carrier_data

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_CARRIER,MKT_CARRIER_FL_NUM,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_DELAY_NEW,CRS_ELAPSED_TIME,DISTANCE,origin-lat-long,origin-tz,origin-elevation,dest-lat-long,dest-tz,dest-elevation,flight_duration,FL_ARR_DATE_REL_ORIGIN,FL_ARR_DATE,takeoff-hour,arriving-hour,takeoff-time-of-day,arrival-time-of-day,ARR_DAY_OF_WEEK,takeoff-congestion-key,arrival-congestion-key,congestion-key,avg-takeoff-congestion,avg-arrival-congestion,dest-congestion-key,dest-avg-takeoff-congestion,dest-avg-arrival-congestion,days-from-holiday,holiday,days-from-specific-holiday,takeoff-mins-from-midnight,landing-mins-from-midnight
0,2021,6,1,Tuesday,2021-06-01 00:15:00,DL,2336,DL,N683DA,2336,LAS,MSP,00:15,1.0,1.0,05:21,1.0,186.0,1299.0,"36.08010101,-115.1520004",America/Los_Angeles,2181.0,"44.8819999695,-93.2218017578",America/Chicago,841.0,0 days 03:06:00,2021-06-01 03:21:00-07:00,2021-06-01 05:21:00,0,5,Night,Early Morning,Tuesday,LASTuesdayNight,MSPTuesdayEarly Morning,LASTuesdayNight,1.329932,1.209226,MSPTuesdayEarly Morning,1.776888,1.383135,1.0,Not a Holiday,Memorial Day_1.0,15,321
1,2021,6,1,Tuesday,2021-06-01 00:15:00,DL,373,DL,N593NW,373,LAX,ATL,00:15,2.0,2.0,07:36,0.0,261.0,1947.0,"33.94250107,-118.4079971",America/Los_Angeles,125.0,"33.6366996765,-84.4281005859",America/New_York,1026.0,0 days 04:21:00,2021-06-01 04:36:00-07:00,2021-06-01 07:36:00,0,7,Night,Early Morning,Tuesday,LAXTuesdayNight,ATLTuesdayEarly Morning,LAXTuesdayNight,1.500295,1.304196,ATLTuesdayEarly Morning,2.732426,1.685451,1.0,Not a Holiday,Memorial Day_1.0,15,456
2,2021,6,1,Tuesday,2021-06-01 00:16:00,UA,505,UA,N27509,505,SFO,IAH,00:16,-9.0,0.0,06:01,0.0,225.0,1635.0,"37.6189994812,-122.375",America/Los_Angeles,13.0,"29.9843997955,-95.3414001465",America/Chicago,97.0,0 days 03:45:00,2021-06-01 04:01:00-07:00,2021-06-01 06:01:00,0,6,Night,Early Morning,Tuesday,SFOTuesdayNight,IAHTuesdayEarly Morning,SFOTuesdayNight,1.498769,1.200000,IAHTuesdayEarly Morning,1.641243,1.719801,1.0,Not a Holiday,Memorial Day_1.0,16,361
3,2021,6,1,Tuesday,2021-06-01 00:20:00,AS,2078,QX,N632QX,2078,ANC,PDX,00:20,-1.0,0.0,05:00,3.0,220.0,1542.0,"61.1744003296,-149.9960021973",America/Anchorage,152.0,"45.58869934,-122.5979996",America/Los_Angeles,31.0,0 days 03:40:00,2021-06-01 04:00:00-08:00,2021-06-01 05:00:00,0,5,Night,Early Morning,Tuesday,ANCTuesdayNight,PDXTuesdayEarly Morning,ANCTuesdayNight,1.032206,1.030812,PDXTuesdayEarly Morning,1.603154,1.063465,1.0,Not a Holiday,Memorial Day_1.0,20,300
4,2021,6,1,Tuesday,2021-06-01 00:25:00,AA,2415,AA,N163US,2415,LAS,CLT,00:25,-6.0,0.0,07:46,0.0,261.0,1916.0,"36.08010101,-115.1520004",America/Los_Angeles,2181.0,"35.2140007019,-80.9430999756",America/New_York,748.0,0 days 04:21:00,2021-06-01 04:46:00-07:00,2021-06-01 07:46:00,0,7,Night,Early Morning,Tuesday,LASTuesdayNight,CLTTuesdayEarly Morning,LASTuesdayNight,1.329932,1.209226,CLTTuesdayEarly Morning,1.708704,1.701318,1.0,Not a Holiday,Memorial Day_1.0,25,466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5744446,2022,6,30,Thursday,2022-06-30 23:59:00,DL,1278,DL,N915DU,1278,ATL,GSP,23:59,11.0,11.0,00:55,1.0,56.0,153.0,"33.6366996765,-84.4281005859",America/New_York,1026.0,"34.8956985474,-82.2189025879",America/New_York,964.0,0 days 00:56:00,2022-07-01 00:55:00-04:00,2022-07-01 00:55:00,23,0,Night,Night,Friday,ATLThursdayNight,GSPFridayNight,ATLThursdayNight,2.385637,1.255467,GSPFridayNight,1.000000,1.021077,-4.0,Not a Holiday,July 4th_-4.0,1439,55
5744447,2022,6,30,Thursday,2022-06-30 23:59:00,UA,531,UA,N37530,531,ANC,ORD,23:59,-3.0,0.0,09:09,0.0,370.0,2846.0,"61.1744003296,-149.9960021973",America/Anchorage,152.0,"41.97859955,-87.90480042",America/Chicago,672.0,0 days 06:10:00,2022-07-01 06:09:00-08:00,2022-07-01 09:09:00,23,9,Night,Late Morning,Friday,AN

In [49]:
grouped_data = carrier_data.groupby(['FL_DATE', 'ORIGIN', 'holiday', 'DAY_OF_WEEK', 'takeoff-time-of-day', 'severe_delay'], as_index=False).size()

Let's preview our data

In [10]:
grouped_data.head()

,FL_DATE_LOCAL,rounded-hour,ORIGIN,holiday,DAY_OF_WEEK,Time of Day,severe_delay,size
0,2021-06-01,0,ANC,Not a Holiday,Tuesday,Night,No Severe Delays,3
1,2021-06-01,0,BZN,Not a Holiday,Tuesday,Night,No Severe Delays,1
2,2021-06-01,0,DEN,Not a Holiday,Tuesday,Night,No Severe Delays,5
3,2021-06-01,0,FLL,Not a Holiday,Tuesday,Night,No Severe Delays,1
4,2021-06-01,0,FLL,Not a Holiday,Tuesday,Night,Severe Delays,1


We'll want the user to be able to graph data on 4 levels:
1. All delays
2. Delays by airport
3. Delays by airline
4. Delays by airline & airport

So we work to pivot our data and save out files for these 4 levels.

In [53]:
# Delays by airport
df_by_airport = pd.pivot_table(grouped_data, values='size', index=['FL_DATE', 'ORIGIN'],
                    columns=['severe_delay'], aggfunc=np.sum, fill_value=0)
df_by_airport = df_by_airport.reset_index()
df_by_airport['percent-delayed'] = df_by_airport['Severe Delays'] / (df_by_airport['No Severe Delays'] + df_by_airport['Severe Delays'])

# Delays by holiday
df_by_holiday = pd.pivot_table(grouped_data, values='size', index=['holiday', 'ORIGIN'],
                    columns=['severe_delay'], aggfunc=np.sum, fill_value=0)
df_by_holiday = df_by_holiday.reset_index()
df_by_holiday['percent-delayed'] = df_by_holiday['Severe Delays'] / (df_by_holiday['No Severe Delays'] + df_by_holiday['Severe Delays'])

# Delays by holiday
df_by_timeofday_weekday = pd.pivot_table(grouped_data, values='size', index=['DAY_OF_WEEK', 'takeoff-time-of-day','ORIGIN'],
                    columns=['severe_delay'], aggfunc=np.sum, fill_value=0)
df_by_timeofday_weekday = df_by_timeofday_weekday.reset_index()
df_by_timeofday_weekday['percent-delayed'] =df_by_timeofday_weekday['Severe Delays'] / (df_by_timeofday_weekday['No Severe Delays'] + df_by_timeofday_weekday['Severe Delays'])

Now let's save these out to files.

In [54]:
df_by_airport.to_csv('data/prepared/delays-by-airport.csv', index=False)
df_by_holiday.to_csv('data/prepared/delays-by-holiday.csv', index=False)
df_by_timeofday_weekday.to_csv('data/prepared/df_by_timeofday_weekday.csv', index=False)